### Car Data Import and Cleanup

We begin by importing all of the data and reviewing descriptive statistics. 

In [39]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np

#Changing the location of this to maintain portability
#kick_data = pd.read_csv('/Users/chandlervaughn/Dropbox/4. Chandler/Development/Machine Learning/Lab1/kick.csv')
kick_data=pd.read_csv("https://www.dropbox.com/s/q2h8uypvc0et5vj/kick.csv?dl=1")

### 1.3.2 Data Cleanup and Imputation

In [40]:
#drop columns that have too little information to be useful
try:
    kick_data = kick_data.drop(['PRIMEUNIT', 'AUCGUART'],axis=1)
except:
    print("Columns not found. Drop has already been performed.")


#should be floats
continuous_features = ['VehOdo', 'MMRAcquisitionAuctionAveragePrice',
                      'MMRAcquisitionAuctionCleanPrice',
                      'MMRAcquisitionRetailAveragePrice',
                      'MMRAcquisitionRetailCleanPrice',
                      'MMRCurrentAuctionAveragePrice',
                      'MMRCurrentAuctionCleanPrice',
                      'MMRCurrentRetailAveragePrice',
                      'MMRCurrentRetailCleanPrice',
                      'VehBCost',
                      'WarrantyCost']

#should be int
ordinal_features = ['IsBadBuy', 'PurchDate', 'VehYear',
                    'VehicleAge', 'WheelTypeID', 'BYRNO',
                    'VNZIP1', 'IsOnlineSale']

#should be categorical
categorical_features = ['Auction', 'Make', 'Model', 'Trim',
                       'SubModel', 'Color', 'Transmission',
                       'WheelType', 'Nationality', 'Size',
                       'TopThreeAmericanName',# 'PRIMEUNIT','AUCGUART',
                        'VNST']

#replace ? data with Nan so we can properly aggregate
kick_data[continuous_features] = kick_data[continuous_features].replace(to_replace='?', value = -9999999)
kick_data[ordinal_features] = kick_data[ordinal_features].replace(to_replace='?', value = -9999999)
kick_data[categorical_features] = kick_data[categorical_features].replace(to_replace='?', value = 'UNKNOWN')

kick_data[continuous_features] = kick_data[continuous_features].astype(np.float64)
kick_data[ordinal_features] = kick_data[ordinal_features].astype(np.int64)

kick_data[continuous_features] = kick_data[continuous_features].replace(to_replace=-9999999, value = np.nan)
kick_data[ordinal_features] = kick_data[ordinal_features].replace(to_replace=-9999999, value = np.nan)

In [41]:
#Impute missing values

kick_mean = kick_data[continuous_features].mean()
kick_median = kick_data[categorical_features+ordinal_features].median()
cat_series = pd.concat((kick_mean, kick_median))
kick_imputed = kick_data.fillna(value=cat_series)

kick_imputed[continuous_features] = kick_imputed[continuous_features].astype(np.float64)
kick_imputed[ordinal_features] = kick_imputed[ordinal_features].astype(np.int64)
#review imputed dataframe
#detail_describe(kick_imputed).transpose()

In [42]:
#------------------------------------------------------------------------------
# accept a dataframe, show outliers
# see http://www.itl.nist.gov/div898/handbook/prc/section1/prc16.htm
#------------------------------------------------------------------------------
def show_outliers(df_in, col_name):
    q1 = df_in[col_name].quantile(0.05)
    q3 = df_in[col_name].quantile(0.95)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = pd.concat([df_in.loc[(df_in[col_name] < fence_low)], df_in.loc[(df_in[col_name] > fence_high)]])
    return df_out


outlier_datapoints = pd.concat([show_outliers(kick_data,'VehBCost'), show_outliers(kick_data,'WarrantyCost')])
outlier_datapoints

,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,...,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
6031,1,1290470400,MANHEIM,2007,3,LEXUS,GS450H,UNKNOWN,4D SEDAN HYBRID,BLACK,...,23400.0,26462.0,29752.0,33337.0,835,85040,AZ,28560.0,0,1121.0
15308,1,1274832000,MANHEIM,2008,2,LEXUS,RX400H AWD,UNKNOWN,4D SPORT UTILITY,GREY,...,31127.0,34798.0,38151.0,41062.0,835,94544,CA,36485.0,0,1001.0
22276,1,1254268800,MANHEIM,2008,1,TOYOTA,HIGHLANDER 4WD V6,Lim,4D SPORT UTILITY HYBRID LIMITED,BLACK,...,35722.0,36859.0,39080.0,40308.0,23359,92807,CA,35900.0,0,702.0
23450,1,1270598400,MANHEIM,2007,3,TOYOTA,FJ CRUISER 4WD V6,UNKNOWN,2D SPORT UTILITY,MAROON,...,18837.0,20348.0,22679.0,25175.0,23359,92807,CA,20380.0,0,522.0
37050,1,1276646400,MANHEIM,2007,3,INFINITI,M35,Bas,4D SEDAN,GREY,...,21940.0,24293.0,27269.0,30194.0,19064,75236,TX,20100.0,0,1020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72432,1,1234915200,ADESA,2003,6,CHEVROLET,VENTURE FWD V6 3.4L,LS,PASSENGER 3.4L,MAROON,...,3945.0,4678.0,4761.0,5552.0,18881,30212,GA,4000.0,0,5392.0
72524,1,1236643200,ADESA,2002,7,CHEVROLET,VENTURE FWD V6 3.4L,LS,PASSENGER EXT 3.4L WARNER BROS,BLACK,...,2821.0,3460.0,3547.0,4237.0,18111,30212,GA,4750.0,0,4922.0
72604,0,1238544000,ADESA,2002,7,PONTIAC,MONTANA 3.4L V6 EFI,UNKNOWN,4D MINIVAN 3.4L VALUE,WHITE,...,2591.0,3352.0,3298.0,4120.0,18111,30212,GA,3500.0,0,5913.0
72903,0,1254355200,ADESA,2004,5,CHEVROLET,VENTURE FWD V6 3.4L,LS,PASSENGER 3.4L,GREEN,...,2109.0,3469.0,2778.0,4247.0,18111,30212,GA,4200.0,0,5613.0


### 1.3.3 Standardization - Rescaling All Continuous Variables

In [43]:
# add mileage (added 9-27-19)
kick_imputed.VehOdo=kick_data.VehOdo.fillna("0")
kick_imputed.VehicleAge=kick_data.VehicleAge.fillna("1")
col1 = 'VehOdo'
col2 = 'VehicleAge'
conditions  = [ (kick_imputed[col1]/kick_imputed[col2]) < 10000, ((kick_imputed[col1]/kick_imputed[col2]) < 13867) & ((kick_imputed[col1]/kick_imputed[col2]) > 10000), (kick_imputed[col1]/kick_imputed[col2]) > 13867 ]
choices     = [ "LOW", 'GOOD', 'HIGH' ]
kick_imputed["Mileage"] = np.select(conditions, choices, default=np.nan)

In [44]:
#create a max/min scaler
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
kick_imputed_scaled = kick_imputed
kick_imputed_scaled[continuous_features] = min_max_scaler.fit_transform(kick_imputed_scaled[continuous_features])

#Dropping correlated values
kick_imputed_scaled = kick_imputed_scaled.drop(['MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailCleanPrice'], axis=1)
kick_imputed_scaled = kick_imputed_scaled.drop(['MMRCurrentAuctionAveragePrice', 'MMRCurrentRetailAveragePrice'], axis=1)
kick_imputed_scaled = kick_imputed_scaled.drop(['MMRCurrentRetailCleanPrice', 'MMRCurrentAuctionCleanPrice'], axis=1)

#detail_describe(kick_imputed_scaled).transpose()

### 1.3.4 Standardized Visualizations

In [45]:
#Imports for Visualization
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline

### 1.3.5 Standardized and Clean Data Profiling

In [46]:
# Kick Rate by Make and Vehicle Year
kick_imputed_grouped = kick_imputed.groupby(by=['Make','VehYear'])
#print ("Kick Rate Broken down by Make and Year:")
#print (kick_imputed_grouped.IsBadBuy.sum() / kick_imputed_grouped.IsBadBuy.count() *100)

## 1.8 New Features

<font color='red'>*** [5 points] Are there other features that could be added to the data or created from
existing features? Which ones?***</font> 


Yes, there are other features that can be added to the data or created from this dataset that can add value in predicting a kick car or 
determining the selling price for a used vehicle purchased at auction.  We will examine a list of features that can be created from the existing data set, new features that can be added from outside data, and a list of add-ons that we would request from Carvana, the owners of this datset if were employed by them to perform this analysis.        


### 1.8.1 Features that can be created from the existing dataset

 - Luxury or Standard vehicle
 
One of the topics discussed regarding kicks were the potential issues an owner faces as a car ages.  Luxury features in a car can be a strong selling tool but they also present costly repairs to the dealership when purchasing from an auction which does not allow for proper vetting of the vehicle. More high tech features and technology can offer a larger range of problems. We researched J.D. Powers, US News and World Report ( https://cars.usnews.com/cars-trucks/best-cars-blog/2016/09/whats-makes-a-luxury-car  and JD Powers (https://www.jdpower.com/cars/luxury-cars/10-most-popular-luxury-cars) to determine see if any of our car "Makes" were identified on the luxury list.   We found "Lexus", "Cadillac", "Lincoln" and "Infiniti" from our dataset were identified as luxury automakers.  
          
A new field called "Luxury" was created with an ordinal value of 1 or 0 if the Make was one of the above automakers.  We were interested in noting if there was a difference in the kick percentages for that group of cars compared to the standard cars.  

In [47]:
#Compare the Make field against the list of previously identified luxury automakers in this dataset.    
kick_imputed_scaled.columns = kick_imputed_scaled.columns.str.replace(' ', '')
temp=kick_imputed_scaled.Make.fillna("0")
kick_imputed_scaled['Luxury'] = pd.np.where(temp.str.contains("LEXUS"),1,
                   pd.np.where(temp.str.contains("CADILLAC"), 1,
                   pd.np.where(temp.str.contains("LINCOLN"), 1, 
                   pd.np.where(temp.str.contains("INFINITI"), 1, 0))))   
 
#Create a crosstab report of the Luxury field and the IsBadBuy (kick identifier) field.  
#luxuryVal_count = pd.crosstab([kick_imputed_scaled['Luxury']], kick_imputed_scaled.IsBadBuy.astype(bool))

#Plot the results
#plt.rcParams["axes.labelsize"] = 16
#plt.rcParams["xtick.labelsize"]= 16
#plt.rcParams["ytick.labelsize"]= 16
#bars = ('Standard', 'Luxury')

#kick_rate_by_luxury = luxuryVal_count.div(luxuryVal_count.sum(1).astype(float),axis=0)
#kick_rate_by_luxury.plot(kind='bar', stacked=True)
#plt.title("Luxury and Standard Vehichles by Buy Type", fontsize=20)

From the graph above you can see that about 30% of the Luxury vehicles are kicks as compared to the standard vehicles.  

 - Vehicle Axles 
 
Vehicle Axles was not specified as a separate field in the dataset.   We created this field from the existing Model field.  
We wanted to see if the axle type reveled any interesting information in this dataset.   The Axle type could be  4WD, FWD, 2WD, AWD.  

In [48]:
#plt.rcParams["axes.labelsize"] = 16
#plt.rcParams["xtick.labelsize"]= 16
#plt.rcParams["ytick.labelsize"]= 16

temp1=kick_imputed_scaled.Model.fillna("0")
kick_imputed_scaled['Axle'] = pd.np.where(temp1.str.contains("2WD"),"2WD",
                     pd.np.where(temp1.str.contains("4WD"), "4WD",
                     pd.np.where(temp1.str.contains("FWD"), "FWD",
                     pd.np.where(temp1.str.contains("AWD"), "AWD", "Unknown"))))


#axleVal_count = pd.crosstab([kick_imputed['Axle']], kick_imputed.IsBadBuy.astype(bool))
#axleVal_count.plot(kind='bar', stacked=True)
#plt.title("Vehicles by Axle vs Bad Buy", fontsize=20)
#plt.show()
#fig= plt.figure(figsize=(8,8))
#kick_rate_by_axle = axleVal_count.div(axleVal_count.sum(1).astype(float),axis=0)
#kick_rate_by_axle.plot(kind='bar', stacked=True)
#plt.title("Percent of Bad Buy by Axle Type", fontsize=20)

The results above shows that the majority of the dataset does not contain the Axle information.   For the small part that is included, 
the AWD and FWD show a larger percentage of kicks than the other categories.  

  - Low, Medium, High Miles
    
One of the major concerns regarding auction vehicles has to do with odometer tampering.  Lowering the miles on the odometer can cause the car to sell for a higher price.  Physical inspection of the car is collectively reviewed with the odometer readings to trigger suspicion of tampering.  THe United States Department of Transportation Federal Highway Administration reports that Americans drive 13,467 miles per year. https://www.autogravity.com/autogravitas/money/whats-average-miles-driven-per-year-car-lease-guide
Using this ranking we assigned a ranking of LOW, GOOD, HIGH to each vehicle based on the odometer reading and the vehicle's age to see how it fared against this rating.  Any mileage less than 10000 miles per year is considered low mileage in the car industry.   The results were interesting.  

### 1.8.2 New Features That Can Be Added From Outside Data

 - City
 - State
    
The physical location of the car can sometimes lead to additional maintenance and corrosion. The Zip codes and State were include in the dataset but the city associated with that zip code was not included.  We downloaded the zip code database 
and created the City field from a merging of the datasets.  The intent was see if the City was in a flood zone like Houston, New Orleans or other coastal cities along the Gulf of Mexico or Florida.   To show the relationships that we are exploring we are showing the data by state below. 


In [49]:
#plt.show()
zip = pd.read_csv('https://www.dropbox.com/s/xg0o3dscrgcp732/uszips.csv?dl=1')
zip=zip[['state_id', 'zip']]
zip.rename(columns = {'state_id' : 'state'}, inplace=True)

kick_imputed_scaled_2 = pd.merge(kick_imputed_scaled, zip, left_on='VNZIP1', right_on='zip', how='left')
kick_imputed_scaled = kick_imputed_scaled_2


#city_count = pd.crosstab([kick_imputed_scaled['state']], kick_imputed_scaled.IsBadBuy.astype(bool))
#city_count.plot(kind='bar', stacked=True, figsize=(20,20))



In [50]:
#add cylinder (added 9-27-19)
temp=kick_imputed_scaled.Model.fillna("0")
kick_imputed_scaled['Cylinder'] = pd.np.where(temp.str.contains("V6"),"V6",
                   pd.np.where(temp.str.contains("6C"), "V6",
                   pd.np.where(temp.str.contains("4C"), "V4", 
                   pd.np.where(temp.str.contains("8C"), "V8",            
                   pd.np.where(temp.str.contains("V4"), "V4",
                   pd.np.where(temp.str.contains("V8"), "V8", "Unknown"))))))

In [51]:
#add model short name (added 9-27-19)
model_short=pd.read_csv("https://www.dropbox.com/s/jlmmghg43x7c1o8/ModelNameDirectory.csv?dl=1")
model_unique = model_short.ShortModelName.unique()

for idx in kick_imputed_scaled.index:
    value = kick_imputed_scaled.loc[idx, "Model"]
    for sub in model_unique:
        if value.find(sub) > -1:
            kick_imputed_scaled.loc[idx,'model_short'] = sub



In [52]:
kick_imputed_scaled['model_short'] = kick_imputed_scaled['model_short'].replace(to_replace=np.nan, value = 'UNKNOWN')

In [53]:
try:
    kick_imputed_scaled = kick_imputed_scaled.drop(['Model'], axis=1)
except:
    print("Columns not found. Drop has already been performed.")


In [54]:
kick_imputed_scaled.transpose()

,0,1,2,3,4,5,6,7,8,9,...,72973,72974,72975,72976,72977,72978,72979,72980,72981,72982
IsBadBuy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
PurchDate,1260144000,1260144000,1260144000,1260144000,1260144000,1260144000,1260144000,1260144000,1260144000,1260144000,...,1259020800,1259712000,1259712000,1259712000,1259712000,1259712000,1259712000,1259712000,1259712000,1259712000
Auction,ADESA,ADESA,ADESA,ADESA,ADESA,ADESA,ADESA,ADESA,ADESA,ADESA,...,ADESA,ADESA,ADESA,ADESA,ADESA,ADESA,ADESA,ADESA,ADESA,ADESA
VehYear,2006,2004,2005,2004,2005,2004,2004,2005,2007,2007,...,2006,2002,2007,2004,2006,2001,2007,2005,2006,2006
VehicleAge,3,5,4,5,4,5,5,4,2,2,...,3,7,2,5,3,8,2,4,3,3
Make,MAZDA,DODGE,DODGE,DODGE,FORD,MITSUBISHI,KIA,FORD,KIA,FORD,...,CHRYSLER,DODGE,HYUNDAI,FORD,KIA,MERCURY,CHEVROLET,JEEP,CHEVROLET,MAZDA
Trim,i,ST,SXT,SXT,ZX3,ES,EX,SE,EX,SEL,...,Bas,ST,Lim,XLS,EX,GS,LS,Lar,LS,s
SubModel,4D SEDAN I,QUAD CAB 4.7L SLT,4D SEDAN SXT FFV,4D SEDAN,2D COUPE ZX3,4D SEDAN ES,4D SEDAN EX,4D SEDAN SE,4D SEDAN EX,4D SEDAN SEL,...,4D SEDAN,QUAD CAB 4.7L SLT,4D SEDAN,4D SUV 4.0L FFV XLS,4D SPORT UTILITY EX,4D SEDAN GS,4D SEDAN LS,4D WAGON LAREDO,4D SEDAN LS,4D SEDAN S
Color,RED,WHITE,MAROON,SILVER,SILVER,WHITE,BLACK,WHITE,BLACK,RED,...,GREEN,GOLD,SILVER,SILVER,GOLD,BLACK,SILVER,SILVER,WHITE,SILVER
Transmission,AUTO,AUTO,AUTO,AUTO,MANUAL,AUTO,AUTO,AUTO,AUTO,AUTO,...,AUTO,AUTO,AUTO,AUTO,AUTO,AUTO,AUTO,AUTO,AUTO,AUTO


In [57]:
#should be categorical
categorical_features = ['Auction', 'Make', 'model_short', 'Trim',
                       'SubModel', 'Color', 'Transmission',
                       'WheelType', 'Nationality', 'Size',
                       'TopThreeAmericanName','Mileage', 'Axle',# 'PRIMEUNIT','AUCGUART',
                        'VNST']

#should be int
ordinal_features = ['IsBadBuy', 'PurchDate', 'VehYear',
                    'VehicleAge', 'WheelTypeID', 'BYRNO',
                    'VNZIP1', 'IsOnlineSale', 'Luxury']


kick_imputed_scaled[ordinal_features] = kick_imputed_scaled[ordinal_features].astype(np.int64)

The zip codes by state show that Florida, Texas and North Carolina have high "kick" counts than other states.  

### 1.8.3 List of Add-ons That We Would Request From The Client 

 - Car Fax Reports 
    
Car Fax Reports can offer additional information that can be used to determine "kicks".  Accident information and number of owners would be additional data that we would like to investigate if the data were available.   Unfortunately, the dataset does not contain the VIN number so we have no way of referencing this history based on the data provided.   


 - Car VIN Number 
    
The VIN number provides an historical tracking for a given car.  Unfortunately, the dataset does not contain the VIN number so we have no way of referencing this history based on the data provided. If we were performing this analysis for our customer, we would request this information.  

In [64]:
import pandas as pd
from pydataset import data
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [72]:
#Lets make dummy variables
one_hot_df = pd.concat([pd.get_dummies(kick_imputed_scaled[col],prefix=col) for col in categorical_features], axis=1)

kick_one_hot=pd.concat([kick_imputed_scaled, one_hot_df], axis=1)

In [73]:
y = kick_one_hot['IsBadBuy']

#drop columns that have too little information to be useful
try:
    kick_one_hot = kick_one_hot.drop(['MMRAcquisitionRetailAveragePrice'], axis=1)
except:
    print("Columns not found. Drop has already been performed.")

try:
    kick_one_hot = kick_one_hot.drop(['IsBadBuy'], axis=1)
except:
    print("Columns not found. Drop has already been performed.")
    
try:
    kick_one_hot = kick_one_hot.drop(['VehYear'], axis=1)
except:
    print("Columns not found. Drop has already been performed.")

#try:
#    kick_one_hot = kick_one_hot.drop(['model_short'], axis=1)
#except:
#    print("Columns not found. Drop has already been performed.")
    
#try:
#    kick_one_hot = kick_one_hot.drop(['model_Short'], axis=1)
#except:
#    print("Columns not found. Drop has already been performed.")

In [74]:
x = kick_one_hot.drop(columns=categorical_features)
#Split the data
#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3,random_state=0)

In [75]:
#clf = LDA(store_covariance=True)
#clf.fit(x_train, y_train)

In [74]:
#y_pred=clf.predict(x_test)

In [75]:
#print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94     19235
           1       0.62      0.29      0.40      2660

    accuracy                           0.89     21895
   macro avg       0.76      0.63      0.67     21895
weighted avg       0.87      0.89      0.87     21895

